# MagnaTagATune Dataset Setup for Jupyter Notebook

**Purpose**: Download and prepare the MagnaTagATune dataset for Amapiano model training

**What this does**:
1. Downloads ~2.9GB of audio from HuggingFace mirror
2. Extracts ~25,863 MP3 files
3. Filters for Amapiano-relevant proxy tags
4. Creates training-ready subset

**Requirements**:
- ~10GB free disk space
- Stable internet connection
- 10-30 minutes depending on download speed

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q pandas tqdm requests

print("✅ Dependencies installed")

## Step 2: Download MagnaTagATune Dataset

This downloads the complete dataset from HuggingFace's mirror (~2.9GB)

In [ ]:
import urllib.request
import zipfile
from pathlib import Path
from tqdm.notebook import tqdm

# Configuration
DATASET_URL = "https://huggingface.co/datasets/confit/magnatagatune/resolve/main/mp3.zip"
DATASET_DIR = Path("../datasets/magnatagatune")
ZIP_PATH = DATASET_DIR / "mp3.zip"
MP3_DIR = DATASET_DIR / "mp3"

DATASET_DIR.mkdir(parents=True, exist_ok=True)

def download_with_progress(url, dest_path):
    """Download file with tqdm progress bar"""
    response = urllib.request.urlopen(url)
    total_size = int(response.headers.get('content-length', 0))
    
    block_size = 8192
    progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc="Downloading")
    
    with open(dest_path, 'wb') as f:
        while True:
            buffer = response.read(block_size)
            if not buffer:
                break
            f.write(buffer)
            progress_bar.update(len(buffer))
    
    progress_bar.close()

# Download
if not ZIP_PATH.exists():
    print("📥 Downloading MagnaTagATune dataset (~2.9GB)...")
    print("This may take 10-30 minutes depending on your connection")
    download_with_progress(DATASET_URL, ZIP_PATH)
    print(f"✅ Downloaded to {ZIP_PATH}")
else:
    print(f"✅ Dataset already downloaded at {ZIP_PATH}")

# Extract
if not MP3_DIR.exists() or not any(MP3_DIR.rglob("*.mp3")):
    print("\n📦 Extracting MP3 files...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(MP3_DIR)
    
    mp3_files = list(MP3_DIR.rglob("*.mp3"))
    print(f"✅ Extracted {len(mp3_files)} MP3 files")
    
    if len(mp3_files) < 20000:
        print(f"⚠️  Expected ~25,863 files, found {len(mp3_files)}")
else:
    mp3_files = list(MP3_DIR.rglob("*.mp3"))
    print(f"✅ MP3 files already extracted ({len(mp3_files)} files)")

# Optional: Clean up ZIP to save space
cleanup = input("\nDelete ZIP file to save ~2.9GB? (y/n): ")
if cleanup.lower() == 'y':
    ZIP_PATH.unlink()
    print("🗑️  ZIP file removed")

## Step 3: Download Annotations

In [ ]:
ANNOTATIONS_URL = "https://mirg.city.ac.uk/datasets/magnatagatune/annotations_final.csv"
ANNOTATIONS_PATH = DATASET_DIR / "annotations_final.csv"

if not ANNOTATIONS_PATH.exists():
    print("📥 Downloading annotations...")
    urllib.request.urlretrieve(ANNOTATIONS_URL, ANNOTATIONS_PATH)
    print(f"✅ Annotations downloaded to {ANNOTATIONS_PATH}")
else:
    print(f"✅ Annotations already exist at {ANNOTATIONS_PATH}")

## Step 4: Load and Explore Annotations

In [ ]:
import pandas as pd

# Load annotations
df = pd.read_csv(ANNOTATIONS_PATH, sep='\t')

print(f"Total clips: {len(df):,}")
print(f"Tag columns: {len(df.columns)}")
print(f"\nFirst few columns: {list(df.columns[:10])}")
print(f"\nDataset shape: {df.shape}")

# Show sample
df.head(3)

## Step 5: Filter for Amapiano Proxy Tags

We filter for tracks with characteristics similar to Amapiano:
- Drums/percussion/beats
- Piano/keyboard/jazz elements  
- Bass/deep basslines
- House/electronic/dance vibes
- Chill/soulful atmosphere

In [ ]:
# Define Amapiano proxy tags
AMAPIANO_PROXY_TAGS = {
    'drums', 'percussion', 'beats', 'techno', 'electronic',
    'piano', 'keyboard', 'jazzy', 'chords', 'jazz',
    'bass', 'bassline', 'deep',
    'house', 'dance', 'groove',
    'chill', 'mellow', 'soulful', 'soft',
    'instrumental', 'ambient'
}

MIN_TAGS_MATCH = 3
TARGET_CLIPS = 8000

# Get tag columns
tag_columns = [col for col in df.columns if col not in ['clip_id', 'mp3_path']]

# Count matching tags for each clip
def count_matching_tags(row):
    matching = 0
    for tag in tag_columns:
        if tag.lower() in AMAPIANO_PROXY_TAGS and row[tag] == 1:
            matching += 1
    return matching

df['amapiano_proxy_score'] = df.apply(count_matching_tags, axis=1)

# Filter and sort
filtered = df[df['amapiano_proxy_score'] >= MIN_TAGS_MATCH].copy()
filtered = filtered.sort_values('amapiano_proxy_score', ascending=False)

print(f"\n✅ Filtered to {len(filtered):,} clips with ≥{MIN_TAGS_MATCH} Amapiano proxy tags")
print(f"\nScore distribution:")
print(filtered['amapiano_proxy_score'].value_counts().sort_index(ascending=False))

# Limit to target
if len(filtered) > TARGET_CLIPS:
    filtered = filtered.head(TARGET_CLIPS)
    print(f"\n✂️  Limited to top {TARGET_CLIPS} clips by proxy score")

filtered.head(5)

## Step 6: Create Training Subset

Copy filtered audio files to a dedicated training directory

In [ ]:
import shutil

OUTPUT_DIR = Path("../datasets/amapiano_proxy")
AUDIO_DIR = OUTPUT_DIR / "audio"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
AUDIO_DIR.mkdir(exist_ok=True)

# Build file lookup map
all_mp3s = {f.name: f for f in MP3_DIR.rglob("*.mp3")}
print(f"Built lookup map for {len(all_mp3s):,} MP3 files")

# Copy files
metadata_rows = []
copied = 0
skipped = 0

print(f"\n📋 Copying {len(filtered)} audio files...")

for idx, row in tqdm(filtered.iterrows(), total=len(filtered), desc="Copying"):
    clip_id = row.get('clip_id', idx)
    mp3_path = row.get('mp3_path', f"{clip_id}.mp3")
    mp3_filename = Path(mp3_path).name
    
    # Find source file
    source_file = MP3_DIR / mp3_path
    if not source_file.exists() and mp3_filename in all_mp3s:
        source_file = all_mp3s[mp3_filename]
    
    if source_file.exists():
        dest_file = AUDIO_DIR / f"{clip_id}.mp3"
        shutil.copy2(source_file, dest_file)
        
        # Get active tags
        tag_list = [col for col in tag_columns if row[col] == 1]
        
        metadata_rows.append({
            'clip_id': clip_id,
            'file_path': str(dest_file.relative_to(OUTPUT_DIR)),
            'amapiano_proxy_score': row['amapiano_proxy_score'],
            'tags': ','.join(tag_list),
            'duration': 29.0
        })
        copied += 1
    else:
        skipped += 1

# Save metadata
metadata_df = pd.DataFrame(metadata_rows)
metadata_path = OUTPUT_DIR / "training_metadata.csv"
metadata_df.to_csv(metadata_path, index=False)

print(f"\n✅ Training subset created:")
print(f"   📁 {copied:,} audio files copied")
print(f"   ⏭️  {skipped:,} files skipped (not found)")
print(f"   📊 Metadata: {metadata_path}")
print(f"   ⏱️  Total duration: {copied * 29 / 3600:.1f} hours")
print(f"   💾 Dataset size: ~{copied * 1.2 / 1024:.1f} GB")

## Step 7: Dataset Statistics

In [ ]:
from collections import Counter

print("=" * 60)
print("AMAPIANO PROXY TRAINING DATASET STATISTICS")
print("=" * 60)
print(f"Total clips: {len(metadata_df):,}")
print(f"Estimated duration: {len(metadata_df) * 29 / 3600:.1f} hours")

print(f"\nProxy score distribution:")
print(metadata_df['amapiano_proxy_score'].value_counts().sort_index(ascending=False))

# Tag frequency
all_tags = []
for tags_str in metadata_df['tags']:
    all_tags.extend(tags_str.split(','))

tag_counts = Counter(all_tags)

print(f"\nTop 20 most common tags:")
for tag, count in tag_counts.most_common(20):
    print(f"  {tag}: {count:,} ({count/len(metadata_df)*100:.1f}%)")

print("=" * 60)

## Step 8: Sample Audio Preview (Optional)

In [ ]:
import IPython.display as ipd
import random

# Pick a random high-scoring clip
high_score = metadata_df[metadata_df['amapiano_proxy_score'] >= 5].sample(1).iloc[0]

print(f"Sample clip: {high_score['clip_id']}")
print(f"Proxy score: {high_score['amapiano_proxy_score']}")
print(f"Tags: {high_score['tags']}")

audio_path = OUTPUT_DIR / high_score['file_path']
display(ipd.Audio(str(audio_path)))

## ✅ Complete!

Your Amapiano proxy training dataset is ready at:
```
../datasets/amapiano_proxy/
├── audio/           # MP3 files
└── training_metadata.csv  # Training metadata
```

**Next steps**:
1. Use this dataset with `train_musicgen.py` for fine-tuning
2. Optionally add real Amapiano samples for better quality
3. Run training and evaluate results